# Tutorial of network dynamics - BioModels
The network state-transition-graph (STG), controlled STG (C-STG) and controlled atttractor graph (CAG) for biological relevant boolean network models

In [2]:
%matplotlib inline

In [3]:
import graphviz
import math
import matplotlib.pyplot as plt
#
import cana
from cana.datasets.bio import THALIANA #, DROSOPHILA, BUDDING_YEAST

In [4]:
N = THALIANA()
#N = DROSOPHILA()
#N = BUDDING_YEAST()
print(N)

<BNetwork(name='Arabidopsis Thaliana', N=15, Nodes=['AP3', 'UFO', 'FUL', 'FT', 'AP1', 'EMF1', 'LFY', 'AP2', 'WUS', 'AG', 'LUG', 'CLF', 'TFL1', 'PI', 'SEP'])>


In [13]:
STG = N.state_transition_graph()
attractors = N.attractors(mode='stg')
print(STG)
print(attractors)
print('Nodes: %d | Edges: %d' % (len(STG.nodes()) , len(STG.edges()) ))

STG: Arabidopsis Thaliana
[[1643], [865], [132], [148], [7787], [7011], [2180], [2196], [5739], [4963]]
Nodes: 8192 | Edges: 8192


In [12]:
if N.name == 'Arabidopsis Thaliana':
    foldername = 'thaliana'
    DNnames = ['UFO','LUG','CLF','SEP','TFL1']
    DNi = [i for i,n in enumerate(N.nodes) if n.name in DNnames]

# Note: CANA will issue a warning for the constant input variables LUG and CLF as these have no inputs
CSTG = N.controlled_state_transition_graph(driver_nodes=DNi)
print( CSTG)
print('Nodes: {:d} | Edges: {:d}'.format(len(CSTG.nodes()) , len(CSTG.edges()) ))

C-STG: Arabidopsis Thaliana (UFO,LUG,CLF,TFL1,SEP)
Nodes: 8192 | Edges: 73701


In [20]:
CAG = N.controlled_attractor_graph(CSTG)
print(CAG)
print('Nodes: {:d} | Edges: {:d}'.format(len(CAG.nodes()) , len(CAG.edges()) ))

IndexError: list index out of range

In [14]:
# Node Position for each one of the models
att = {}
if N.name == 'Arabidopsis Thaliana':
    foldername = 'thaliana'
    
    # Calculates Node position in a circle. Used to plot nodes always in the same position.
    for deg,name in zip(range(0,360,30), ['AG', 'AP3', 'PI', 'AP2', 'TFL1', 'FUL', 'AP1', 'FT', 'EMF1', 'LFY', 'SEP', 'WUS']):
        r = 120
        x, y = r*math.cos(math.radians(deg)), r*math.sin(math.radians(deg))
        att[name] = {'x':x,'y':y}
    # Nodes not on the circle. Manually position them = UFO, LUG, CLF
    for name,(x,y) in zip(['UFO','LUG','CLF'], [(160.,90.),(180.,30.),(180.,-30.)]):
        att[name] = {'x':x,'y':y}

In [15]:
# Draw the Controlled Attractor Graph
S = graphviz.Digraph(name='Control State Transition Graph', engine='neato')
S.attr('graph', concentrate='true', simplify='false', overlap='false',splines='true')
S.attr('node', shape='doublecircle', fixedsize='true', width='0.5', style='filled', penwidth='3', fontname='Helvetica', fontcolor='black',fontsize='14')
S.attr('edge', arrowhead='normal', style='solid', arrowsize='.5')

for node,d in CAG.nodes(data=True):
    if node in attractors:
        fillcolor = 'gray'
    else:
        xy = 'false'
        fillcolor = 'white'
    S.node(name=str(node), label=str(node+1), fillcolor=fillcolor, color='#0000b2')

for s, t, d in CAG.edges(data=True):
    if (s,t) not in att:
        color='#cc0000'
    else:
        color='gray'
    S.edge(str(s),str(t), weight='1', color=color ,len='1.5', penwidth='2.')

# Display
display(S)

NameError: name 'CAG' is not defined

In [17]:
Nsg = N.structural_graph()

In [18]:
# Draw a Specific Attractor Configuration
n = 5 # [3, 5]
configuration = CAG.node[n-1]['label']

S = graphviz.Digraph(name='Structural Graph', engine='neato')
S.attr('graph', concentrate='false', simplify='false', overlap='false',splines='true')
S.attr('node', shape='circle', fixedsize='true', width='.5', style='filled', penwidth='4', fontname='Helvetica', fontsize='12')
S.attr('edge', arrowhead='normal', arrowsize='.5', color='#545454')

for state,(node,d) in zip(configuration, Nsg.nodes(data=True)):
    if d['label'] in att:
        natt = att[d['label']]
        if 'x' in natt or 'y' in natt:
            x,y = natt['x'] , natt['y']
            xy = '%.2f,%.2f!' % (x/72,y/72)
    else:
        xy = 'false'
    if node in DNi:
        color = '#ffa500'
    else:
        color = 'gray'
    if state=='1':
        fontcolor ='white'
        fillcolor = 'black'
    else:
        fontcolor ='black'
        fillcolor = 'white'
    S.node(name=str(node), label=d['label'], pos=xy, color=color, fillcolor=fillcolor, fontcolor=fontcolor)

max_penwidth = 2.
for s,t,d in Nsg.edges(data=True):
    weight = '%d' % (d['weight']*100)
    penwidth_scaled = '%.2f' % ( (d['weight']/1)*max_penwidth )
    S.edge(str(s),str(t), weight=weight, penwidth=penwidth_scaled, )

## Display
display(S)

NameError: name 'CAG' is not defined

In [12]:
# Calculate Effective Graph
threshold = 0.00
Neg = N.effective_graph(mode='input',bound='upper', threshold=threshold)

In [19]:
# Draw a Specific Attractor Configuration
n = 3 # [3, 5]
configuration = CAG.node[n-1]['label']
configuration = CAG.node[n-1]['label']

S = graphviz.Digraph(name='Structural Graph', engine='neato')
S.attr('graph', concentrate='false', simplify='false', overlap='false',splines='true')
S.attr('node', shape='circle', fixedsize='true', width='.5', style='filled', penwidth='4', fontname='Helvetica', fontsize='12')
S.attr('edge', arrowhead='normal', arrowsize='.5', color='#545454')

for state,(node,d) in zip(configuration, Neg.nodes(data=True)):
    if d['label'] in att:
        natt = att[d['label']]
        if 'x' in natt or 'y' in natt:
            x,y = natt['x'] , natt['y']
            xy = '%.2f,%.2f!' % (x/72,y/72)
    else:
        xy = 'false'
    if node in DNi:
        color = '#ffa500'
    else:
        color = 'gray'
    if state=='1':
        fontcolor ='white'
        fillcolor = 'black'
    else:
        fontcolor ='black'
        fillcolor = 'white'
    S.node(name=str(node), label=d['label'], pos=xy, color=color, fillcolor=fillcolor, fontcolor=fontcolor)

max_penwidth = 2.
for s,t,d in Neg.edges(data=True):
    weight = '%d' % (d['weight']*100)
    penwidth_scaled = '%.2f' % ( (d['weight']/1)*max_penwidth )
    S.edge(str(s),str(t), weight=weight, penwidth=penwidth_scaled, )
# Display
display(S)

NameError: name 'CAG' is not defined